In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Installing Auto-ts and other libraries

In [ ]:
pip install auto-ts

In [ ]:
pip install xlrd

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
import plotly.express as px
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

# Reading Data

In [ ]:
df=pd.read_csv('../input/ethereum-usd-ethusd/ETH-USD.csv')

In [ ]:
df.info()

In [ ]:
df['Date']=pd.to_datetime(df['Date'])
df.info()

In [ ]:
print ("Rows     : " ,df.shape[0])
print ("Columns  : " ,df.shape[1])
print ("\nFeatures : \n" ,df.columns.tolist())
print ("\nMissing values :  ", df.isnull().any())
print ("\nUnique values :  \n",df.nunique())

In [ ]:
def fill_missing(df):
    ### function to impute missing values using interpolation ###
    df['Open'] = df['Open'].interpolate()
    df['Close'] = df['Close'].interpolate()
    df['Adj Close'] = df['Adj Close'].interpolate()
    df['Volume'] = df['Volume'].interpolate()
    df['High'] = df['High'].interpolate()
    df['Low'] = df['Low'].interpolate()
    print(df.head())
    print(df.isnull().sum())
    
fill_missing(df)

In [ ]:
print ("Rows     : " ,df.shape[0])
print ("Columns  : " ,df.shape[1])
print ("\nFeatures : \n" ,df.columns.tolist())
print ("\nMissing values :  ", df.isnull().any())
print ("\nUnique values :  \n",df.nunique())

# EDA

In [ ]:
fig = px.line(df, x='Date', y='Open', title='Opening price')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=2, label="3y", step="year", stepmode="backward"),
            dict(count=3, label="5y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
fig = px.line(df, x='Date', y='Close', title='Closing price')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=2, label="3y", step="year", stepmode="backward"),
            dict(count=3, label="5y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:

fig = px.line(df, x='Date', y='Volume', title='Volume')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=2, label="3y", step="year", stepmode="backward"),
            dict(count=3, label="5y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
df_indexed=df.set_index('Date')
df_indexed

In [ ]:
df_indexed.resample('M').mean().plot(subplots=True)

In [ ]:
color_list = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
]


def Visualize(data):
    features= list(df.select_dtypes(include=[np.number]).columns.values)
    feature_size=len(features)
    fig, axes = plt.subplots(
        nrows=int(np.ceil(feature_size/2)), ncols=2, figsize=(14, feature_size*2), dpi=80, facecolor="w", edgecolor="k"
    )
    for i in range(feature_size):
        key = features[i]
        c = color_list[i % (len(color_list))]
        t_data = data[key]
        t_data.head()
        ax = t_data.plot(
            ax=axes[i // 2, i % 2],
            color=c,
            title="{}".format(key),
            rot=25,
        )
        ax.legend([key])
    plt.tight_layout()


Visualize(df_indexed)

# Checking that whether data is Stationary or Not using Augmented Dickey-Fuller test

In [ ]:
for i in range(len(df_indexed.columns)):
  result = adfuller(df_indexed[df_indexed.columns[i]])

  if result[1] > 0.05 :
    print('{} - Series is not Stationary'.format(df_indexed.columns[i]))
  else:
    print('{} - Series is Stationary'.format(df_indexed.columns[i]))

# Seasonal Decompose

In [ ]:
import statsmodels.api as sm
res = sm.tsa.seasonal_decompose(df_indexed['Open'],
                                model='multiplicative')

resplot = res.plot()

In [ ]:
import statsmodels.api as sm
res = sm.tsa.seasonal_decompose(df_indexed['Volume'],
                                model='multiplicative')

resplot = res.plot()

In [ ]:
import statsmodels.api as sm
res = sm.tsa.seasonal_decompose(df_indexed['Close'],
                                model='multiplicative')

resplot = res.plot()

# Data Wrangling

In [ ]:
daily_data_fb = df.reset_index()[['Date','Close']].rename({'Date':'ds','Close':'y'}, axis=1)
daily_data_fb.head()

In [ ]:
test_length = 365
split_date = "2020-11-01"
train_filt = daily_data_fb['ds'] <= split_date
test_filt = daily_data_fb['ds'] > split_date

train_fb = daily_data_fb[train_filt]
test_fb = daily_data_fb[test_filt]

print("train data shape :", train_fb.shape[0] / daily_data_fb.shape[0] * 100)
print("test data shape :", test_fb.shape[0] / daily_data_fb.shape[0] * 100)

In [ ]:
from auto_ts import auto_timeseries

In [ ]:
train_fb = train_fb.reset_index()
train_fb.head()

In [ ]:
test_fb = test_fb.reset_index()
test_fb.head()

In [ ]:
automl_model = auto_timeseries(score_type='rmse',
                time_interval='D', forecast_period=30,
                non_seasonal_pdq=None, seasonality=False, seasonal_period=12,
                model_type=['best'],
                verbose=2)

In [ ]:
ts_column = 'ds'
target = 'y'
sep = ','

# Fitting Data in Model

In [ ]:
automl_model.fit(
    traindata=train_fb,
    ts_column=ts_column,
    target=target,
    cv=5,
    sep=',')

In [ ]:
automl_model.get_leaderboard()

In [ ]:
automl_model.get_cv_scores()

In [ ]:
y_pred = automl_model.predict(test_fb,model='best')
predf = pd.DataFrame([test_fb[target].values, y_pred['yhat'].values]).T
predf.columns = ['Actual','Predicted']
predf.head()

In [ ]:
from auto_ts import print_ts_model_stats
print_ts_model_stats(test_fb[target].values, y_pred['yhat'].values,'Best')

In [ ]:
# Using Best Model
future_predictions = automl_model.predict(
    testdata=test_fb
)  
future_predictions

In [ ]:
# Using specific model
future_predictions = automl_model.predict(
    testdata=test_fb,
    model='auto_SARIMAX'
)
future_predictions

In [ ]:
# Using specific model
future_predictions = automl_model.predict(
    testdata=test_fb,
    model='Prophet',
    simple=False

)
future_predictions

In [ ]:
import copy
subm = test_fb[[target]]
print(subm.shape)
subm.head()

In [ ]:
subm['predictions'] = future_predictions['yhat'].values
subm[[target,'predictions']].plot(figsize=(15,8))